<a href="https://colab.research.google.com/github/Hamza-Ali0237/PyTorch-TicketClassification/blob/main/Ticket-Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ticket Classifiction Using CNNs In PyTorch (DataCamp Project)

In [1]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 9.4 MB/s eta 0:00:00


In [2]:
from collections import Counter
import nltk, json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
# Import data and labels
with open("words.json", 'r') as f1:
    words = json.load(f1)
with open("text.json", 'r') as f2:
    text = json.load(f2)
labels = np.load('labels.npy')

In [5]:
# Dictionaries to store the word to index mappings and vice versa
word2idx = {o:i for i,o in enumerate(words)}
idx2word = {i:o for i,o in enumerate(words)}

# Looking up the mapping dictionary and assigning the index to the respective words
for i, sentence in enumerate(text):
    text[i] = [word2idx[word] if word in word2idx else 0 for word in sentence]

# Defining a function that either shortens sentences or pads sentences with 0 to a fixed length
def pad_input(sentences, seq_len):
    features = np.zeros((len(sentences), seq_len),dtype=int)
    for ii, review in enumerate(sentences):
        if len(review) != 0:
            features[ii, -len(review):] = np.array(review)[:seq_len]
    return features

text = pad_input(text, 50)

In [6]:
# Splitting dataset
train_text, test_text, train_label, test_label = train_test_split(text, labels, test_size=0.2, random_state=42)

train_data = TensorDataset(torch.from_numpy(train_text), torch.from_numpy(train_label).long())
test_data = TensorDataset(torch.from_numpy(test_text), torch.from_numpy(test_label).long())

## Start coding here

In [7]:
# Define Model
class CNNClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, target_size):
        super(CNNClassifier, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.conv = nn.Conv1d(embed_dim, embed_dim, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(embed_dim, target_size)

    def forward(self, x):
        embedded = self.embed(x).permute(0,2,1)
        conved = F.relu(self.conv(embedded))
        conved = conved.mean(dim=2)

        return self.fc(conved)

In [8]:
# Instantiate Model
model = CNNClassifier(
    vocab_size = len(word2idx)+1,
    embed_dim = 64,
    target_size = len(np.unique(labels))
)

In [9]:
print(model)

CNNClassifier(
  (embed): Embedding(10147, 64)
  (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (fc): Linear(in_features=64, out_features=5, bias=True)
)


In [10]:
# DataLoader
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [11]:
# Training
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

for epoch in range(3):
    for sentences, labels in train_loader:
        model.zero_grad()
        outputs = model(sentences)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{3}, Loss: {loss.item():.4f}")

Epoch 1/3, Loss: 0.6071
Epoch 2/3, Loss: 0.4651
Epoch 3/3, Loss: 0.1426


In [12]:
# Instantiate Evaluation Metrics
accuracy_metric = Accuracy(task="multiclass", num_classes=5)
precision_metric = Precision(task="multiclass", num_classes=5, average=None)
recall_metric = Recall(task="multiclass", num_classes=5, average=None)